# Activity : Word2Vec embeddings

In the section about Text Classification, we used various approaches to embed the sentences to classify. Some of the reference techniques involved statistical modelling (e.g. BOW, TFIDF), and others involved learning representations for words.

In this activity, we propose to implement Word2Vec embeddings in PyTorch (during the previous activity we used the gensim implementation of the algorithm)

## 1. Corpus

We will work on a very simple corpus :

In [1]:
corpus = [
    'louis est roi de france',
    'marie est reine de france',
    'louis est un homme',
    'marie est une femme',
    'paris est la capitale de la france',
    'bruxelles est la capitale de la belgique',
    'tokyo est la capitale du japon',
]

In [2]:
%pip install unidecode

     |████████████████████████████████| 235 kB 5.5 MB/s 


In [7]:
import nltk
nltk.download('stopwords')
nltk.download('words')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

Using the same implementation as what we did earlier, we propose to preprocess the corpus

In [8]:
from nltk.stem.snowball import FrenchStemmer
from nltk import wordpunct_tokenize          
from nltk.corpus import stopwords
from nltk.corpus import words
from string import punctuation
import unidecode

class FrenchTokenizer(object):
    def __init__(self):
        self.stopwords = set(stopwords.words('french'))
        self.words = set(words.words())
    def __call__(self, doc):
        # tokenize words and punctuation
        word_list = wordpunct_tokenize(doc)
        # remove stopwords
        word_list = [word for word in word_list if word not in self.stopwords]
        # remove 1-character words
        word_list = [word for word in word_list if len(word)>1]
        # remove non alpha
        word_list = [word for word in word_list if word.isalpha()]
        return [unidecode.unidecode(t) for t in word_list]

#Tokenizing sentences
tok=FrenchTokenizer()
text_for_word2vec=[tok(sent) for sent in corpus]

In [9]:
#Vocabulary modelling : assigning indexes to words
vocabulary = []
for sentence in text_for_word2vec:
    for token in sentence:
        if token not in vocabulary:
            vocabulary.append(token)

word2idx = {w: idx for (idx, w) in enumerate(vocabulary)}
idx2word = {idx: w for (idx, w) in enumerate(vocabulary)}

vocabulary_size = len(vocabulary)
print("vocabulary size : ", vocabulary_size)

vocabulary size :  13


The Word2Vec architecture relies on predicting the tokens surrounding a given one (or vice versa). It is composed of a simple auto-encoder using one-hot encodings of the tokens as inputs and outputs, and a hidden layer of a chosen size that will represent the embedding.

![skipgram](img/word2vecskipgram.png)

Because input output vectors are one-hot encodings of the tokens, their dimensionnality is the size of the corpus (e.g : if there are 30 tokens in the vocabulary, the encoding will be of size 30, with each dimension representing the presence or absence of the token in the message)
The hidden layer is of size N. Because of the auto-encoder structure, the hidden layer will be trained to constrain the information about tokens in a space of given dimensionnality.

There are two approaches for Word2Vec :
- the Continuous Bag-of-words (CBOW) appraoach, where the token is used to predict surrounding ones
- the Skip-Gram approach, where context tokens are used to predict the central one

In this exercice, we propose to implement the SkipGram approach. To train this model, we need token pairs (a context token and the central one).
Token pairs are generated by going through all tokens and generating context tokens belonging to a window of a given size around the token.

In [10]:
import numpy as np

window_size = 1

idx_pairs = []
# for each sentence
for sentence in text_for_word2vec:
    indices = [word2idx[word] for word in sentence]
    # for each word, treated as center word
    for center_word_pos in range(len(indices)):
        # for each window position
        for w in range(-window_size, window_size + 1):
            context_word_pos = center_word_pos + w
            # make soure not jump out sentence
            if context_word_pos < 0 or context_word_pos >= len(indices) or center_word_pos == context_word_pos:
                continue
            context_word_idx = indices[context_word_pos]
            idx_pairs.append((indices[center_word_pos], context_word_idx))

idx_pairs = np.array(idx_pairs)

#Printing first 10 pairs
for i in range(10):
    print(idx2word[idx_pairs[i][0]], idx2word[idx_pairs[i][1]])

louis roi
roi louis
roi france
france roi
marie reine
reine marie
reine france
france reine
louis homme
homme louis


To use these pairs to train the network, we need to encode them in a layer :

In [11]:
import torch

def get_layer(word_idx):
    x = torch.zeros(vocabulary_size).float()
    x[word_idx] = 1.0
    return x

#exemple
get_layer(2)

tensor([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

<div class="alert alert-success">
Exercice : 
    
Complete the class below to implement the architecture of the Skipgram.
The class will include a method get_wv to get the word vector (i.e. the hidden layer) for a given word
</div>

In [15]:
#COMPLETE HERE
import torch.nn as nn
import numpy as np

class skipgram(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        #vocab size : vocabulary size (corresponding to input and output dimensions)
        #embedding_dim : dimension of the embedding (hidden) layer
        super(skipgram, self).__init__()
        self.lin1 = nn.Linear(vocab_size, embedding_dim)
        self.lin2 = nn.Linear(embedding_dim, vocab_size)
        self.soft = nn.Softmax(dim=0)

    def forward(self, input):
        hidden = self.lin1(input)
        output = self.soft(self.lin2(hidden))
        return output

    def get_wv(self,input):
        #get the word vector for a given input
        return self.lin1(input).detach().numpy()


Training the network :

In [16]:
model = skipgram(vocab_size=vocabulary_size,embedding_dim=2)
loss_function = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

num_epochs = 100

for epo in range(num_epochs):
    loss_val = 0
    for center_word, target_word in idx_pairs:
        optimizer.zero_grad()
        input = get_layer(center_word)
        target = get_layer(target_word)
        output=model(input)
        loss = loss_function(output, target)
        loss_val += loss.item()
        loss.backward()
        optimizer.step()
    if epo%10==0:
        print(f'Loss at epoch {epo}: {loss_val/len(idx_pairs)}')

Loss at epoch 0: 0.2852527356396119
Loss at epoch 10: 0.2714952913423379
Loss at epoch 20: 0.2624969854950905
Loss at epoch 30: 0.256349578499794
Loss at epoch 40: 0.2517835671703021
Loss at epoch 50: 0.2475977490345637
Loss at epoch 60: 0.2430145125836134
Loss at epoch 70: 0.23764296310643354
Loss at epoch 80: 0.2314088667432467
Loss at epoch 90: 0.224520580843091


Let's have a look at the word embeddings. They could be used to represent individual tokens in a classification task, for instance 

In [17]:
word_vectors = {w : model.get_wv(get_layer(word2idx[w])) for w in vocabulary}

print(word_vectors)

{'louis': array([-0.12456959, -0.38728043], dtype=float32), 'roi': array([ 0.40242693, -0.5989847 ], dtype=float32), 'france': array([-0.7079929, -0.8947082], dtype=float32), 'marie': array([-0.15035853, -0.74644256], dtype=float32), 'reine': array([-0.47818613, -0.02521625], dtype=float32), 'homme': array([ 0.0708158 , -0.25231364], dtype=float32), 'femme': array([-0.5924646,  0.6148186], dtype=float32), 'paris': array([-1.0490847, -0.6064512], dtype=float32), 'capitale': array([ 0.3193592 , -0.24631289], dtype=float32), 'bruxelles': array([-0.9316498, -0.5611542], dtype=float32), 'belgique': array([-1.0585146 , -0.42692173], dtype=float32), 'tokyo': array([-1.0012003, -0.7326194], dtype=float32), 'japon': array([-1.209703  , -0.40730464], dtype=float32)}
